In [ ]:
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt

from mapper_functions import plot_global_tight_pcm
from my_functions import read_obsfcstana_extend_datetime

In [ ]:
# Read /Users/amfox/Desktop/GEOSldas_diagnostics/test_data/land_sweeper/LS_OLv8_M36/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2020/M01/LS_OLv8_M36.ens_avg.ldas_ObsFcstAna.20200102_0300z.bin

path = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/land_sweeper/LS_OLv8_M36/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2020/M01'
file_name_start = 'LS_OLv8_M36.ens_avg.ldas_ObsFcstAna.2020012'
printflag = True

date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana_extend_datetime(path, file_name_start, printflag)
o_date_time = np.array(date_time)
o_obs_species = np.array(obs_species)
o_obs_tilenum = np.array(obs_tilenum)
o_obs_lon = np.array(obs_lon)
o_obs_lat = np.array(obs_lat)
o_obs_obs = np.array(obs_obs)
o_obs_fcst = np.array(obs_fcst)
o_obs_ana = np.array(obs_ana)

missing_val = -9999.0
o_obs_obs[o_obs_obs == missing_val] = np.nan
o_obs_fcst[o_obs_fcst == missing_val] = np.nan
o_obs_ana[o_obs_ana == missing_val] = np.nan



In [ ]:
path = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/land_sweeper/LS_DAv8_M36/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2020/M01'
file_name_start = 'LS_DAv8_M36.ens_avg.ldas_ObsFcstAna.2020012'
printflag = True

date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana_extend_datetime(path, file_name_start, printflag)
d_date_time = np.array(date_time)
d_obs_species = np.array(obs_species)
d_obs_tilenum = np.array(obs_tilenum)
d_obs_lon = np.array(obs_lon)
d_obs_lat = np.array(obs_lat)
d_obs_obs = np.array(obs_obs)
d_obs_fcst = np.array(obs_fcst)
d_obs_ana = np.array(obs_ana)

missing_val = -9999.0
d_obs_obs[d_obs_obs == missing_val] = np.nan
d_obs_fcst[d_obs_fcst == missing_val] = np.nan
d_obs_ana[d_obs_ana == missing_val] = np.nan

print(d_date_time[0])



In [ ]:
import pandas as pd
import numpy as np

# Assuming o_date_time and d_date_time are dictionaries with arrays for each component of the date and time

def parse_date_time(date_time):
    return [
        f"{dt['year'][0]:04d}-{dt['month'][0]:02d}-{dt['day'][0]:02d} {dt['hour'][0]:02d}:{dt['min'][0]:02d}:{dt['sec'][0]:02d}"
        for dt in date_time
    ]

# Parse date_time
o_date_time_parsed = parse_date_time(o_date_time)
d_date_time_parsed = parse_date_time(d_date_time)

# Create DataFrames
o_df = pd.DataFrame({
    'date_time': o_date_time_parsed,
    'species': o_obs_species,
    'tilenum': o_obs_tilenum,
    'lon': o_obs_lon,
    'lat': o_obs_lat
})

d_df = pd.DataFrame({
    'date_time': d_date_time_parsed,
    'species': d_obs_species,
    'tilenum': d_obs_tilenum,
    'lon': d_obs_lon,
    'lat': d_obs_lat
})

# Merge DataFrames on all columns
merged_df = pd.merge(o_df.reset_index(), d_df.reset_index(), on=['date_time', 'species', 'tilenum', 'lon', 'lat'], suffixes=('_o', '_d'))

# Extract indices
o_indices = merged_df['index_o'].values
d_indices = merged_df['index_d'].values

# Convert to numpy arrays
o_indices = np.array(o_indices)
d_indices = np.array(d_indices)

# Apply the indices to the data arrays
o_obs_obs = o_obs_obs[o_indices]
o_obs_species = o_obs_species[o_indices]
o_obs_fcst = o_obs_fcst[o_indices]
o_obs_ana = o_obs_ana[o_indices]

d_obs_obs = d_obs_obs[d_indices]
d_obs_species = d_obs_species[d_indices]
d_obs_fcst = d_obs_fcst[d_indices]
d_obs_ana = d_obs_ana[d_indices]



In [ ]:
i = 5

num_obs = len(o_obs_obs[o_obs_species ==i])
print('Number of OL observations: ', num_obs)

num_obs = len(d_obs_obs[d_obs_species ==i])
print('Number of DA observations: ', num_obs)

o_omf = o_obs_fcst[o_obs_species == i] - o_obs_obs[o_obs_species == i]
d_omf = d_obs_fcst[d_obs_species == i] - d_obs_obs[d_obs_species == i]
b_omf = o_obs_fcst[o_obs_species == i] - d_obs_obs[d_obs_species == i]

o_oma = o_obs_ana[o_obs_species == i] - o_obs_obs[o_obs_species == i]
d_oma = d_obs_ana[d_obs_species == i] - d_obs_obs[d_obs_species == i]
b_oma = o_obs_ana[o_obs_species == i] - d_obs_obs[d_obs_species == i]

o_inc = o_obs_ana[o_obs_species == i] - o_obs_fcst[o_obs_species == i]
d_inc = d_obs_ana[d_obs_species == i] - d_obs_fcst[d_obs_species == i]

# Plot the OMF and OMA
plt.figure()
plt.plot(o_omf, 'ro')
plt.plot(d_omf, 'bo')
plt.plot(b_omf, 'g+')
plt.legend(['OL', 'DA', 'Bias corrected OL'])
plt.title('OMF')

plt.figure()
plt.plot(o_oma, 'ro')
plt.plot(d_oma, 'bo')
plt.plot(b_oma, 'g+')
plt.legend(['OL', 'DA', 'Bias corrected OL'])
plt.title('OMA')

plt.figure()
plt.plot(o_inc, 'ro')
plt.plot(d_inc, 'bo')
plt.legend(['OL', 'DA'])
plt.title('Increment')

In [ ]:
for i in range(1, 14):
    o_omf = o_obs_fcst[o_obs_species == i] - o_obs_obs[o_obs_species == i]
    d_omf = d_obs_fcst[d_obs_species == i] - d_obs_obs[d_obs_species == i]
    b_omf = o_obs_fcst[o_obs_species == i] - d_obs_obs[d_obs_species == i]

    o_oma = o_obs_ana[o_obs_species == i] - o_obs_obs[o_obs_species == i]
    d_oma = d_obs_ana[d_obs_species == i] - d_obs_obs[d_obs_species == i]

    o_inc = o_obs_ana[o_obs_species == i] - o_obs_fcst[o_obs_species == i]
    d_inc = d_obs_ana[d_obs_species == i] - d_obs_fcst[d_obs_species == i]

    print('i = ', i) 

    if np.max(np.abs(o_obs_fcst[o_obs_species == i])) > 1000:
        print('OL OMF has a large value:', np.max(np.abs(o_obs_fcst[o_obs_species == i])))
    if np.max(np.abs(d_obs_fcst[d_obs_species == i])) > 1000:
        print('DA OMF has a large value:', np.max(np.abs(d_obs_fcst[d_obs_species == i])))
    if np.max(np.abs(o_obs_ana[o_obs_species == i])) > 1000:
        print('OL OMA has a large value:', np.max(np.abs(o_obs_ana[o_obs_species == i])))
    if np.max(np.abs(d_obs_ana[d_obs_species == i])) > 1000:
        print('DA OMA has a large value:', np.max(np.abs(d_obs_ana[d_obs_species == i])))

    print('OL OMF mean: ', np.nanmean(o_omf), ' OL OMF std: ', np.nanstd(o_omf))
    print('DA OMF mean: ', np.nanmean(d_omf), ' DA OMF std: ', np.nanstd(d_omf))
    print('BI OMF mean: ', np.nanmean(b_omf), ' BI OMF std: ', np.nanstd(b_omf))
    print('OL OMA mean: ', np.nanmean(o_oma), ' OL OMA std: ', np.nanstd(o_oma), 'Max abs OMA: ', np.nanmax(np.abs(o_oma)))
    print('DA OMA mean: ', np.nanmean(d_oma), ' DA OMA std: ', np.nanstd(d_oma), 'Max abs OMA: ', np.nanmax(np.abs(d_oma)))
    print('OL Increment mean: ', np.nanmean(o_inc), ' OL Increment std: ', np.nanstd(o_inc))
    print('DA Increment mean: ', np.nanmean(d_inc), ' DA Increment std: ', np.nanstd(d_inc))

In [ ]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

expt1_name = 'LS_OLv8_M36'
expt2_name = 'LS_DAv8_M36'

start_date = datetime(2020, 1, 1)
end_date = datetime(2020, 2, 1)

start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

In [ ]:
# Calculate the daily statistics in observation space

# Define the print flag
printflag = False

# Initialize lists to store the mean values for each variable and the dates
dates_list = []

# Initialize daily mean lists for observation space
daily_o_omf_mean = [] 
daily_d_omf_mean = []
daily_b_omf_mean = []
daily_o_oma_mean = []
daily_d_oma_mean = []
daily_o_inc_mean = []
daily_d_inc_mean = []


# Define the start and end dates
# start_date = datetime.strptime('20150401', '%Y%m%d')
# end_date = datetime.strptime('20210331', '%Y%m%d')

# Loop over the dates
current_date = start_date
while current_date <= end_date:

    print('current_date = ', current_date)

    # Define the file name for the current date

    # Define the path directory
    path_dir = f'/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/land_sweeper/{expt1_name}/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg'

    # Define the common file name start
    file_name_start = f'{expt1_name}.ens_avg.ldas_ObsFcstAna.'

    file_name = file_name_start + current_date.strftime('%Y%m%d')
    #if file_name[-4:] == '0105':
    #    print('file_name = ', file_name)
    
    # Call the read_obsfcstana function for the current file
    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana_extend_datetime(path_dir, file_name, printflag)

    # Convert the lists to numpy arrays
    o_date_time = np.array(date_time)
    o_obs_species = np.array(obs_species)
    o_obs_tilenum = np.array(obs_tilenum)
    o_obs_lon = np.array(obs_lon)
    o_obs_lat = np.array(obs_lat)
    o_obs_obs = np.array(obs_obs)
    o_obs_fcst = np.array(obs_fcst)
    o_obs_ana = np.array(obs_ana)

    missing_val = -9999.0
    o_obs_obs[o_obs_obs == missing_val] = np.nan
    o_obs_fcst[o_obs_fcst == missing_val] = np.nan
    o_obs_ana[o_obs_ana == missing_val] = np.nan


    # Call the read_obsfcstana function for the expt2 file
    path_dir = f'/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/land_sweeper/{expt2_name}/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg'

    file_name_start = f'{expt2_name}.ens_avg.ldas_ObsFcstAna.'
    file_name = file_name_start + current_date.strftime('%Y%m%d')

    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana_extend_datetime(path_dir, file_name, printflag)

    # Convert the lists to numpy arrays
    d_date_time = np.array(date_time)
    d_obs_species = np.array(obs_species)
    d_obs_tilenum = np.array(obs_tilenum)
    d_obs_lon = np.array(obs_lon)
    d_obs_lat = np.array(obs_lat)
    d_obs_obs = np.array(obs_obs)
    d_obs_fcst = np.array(obs_fcst)
    d_obs_ana = np.array(obs_ana)

    missing_val = -9999.0
    d_obs_obs[d_obs_obs == missing_val] = np.nan
    d_obs_fcst[d_obs_fcst == missing_val] = np.nan
    d_obs_ana[d_obs_ana == missing_val] = np.nan


    def parse_date_time(date_time):
        return [
            f"{dt['year'][0]:04d}-{dt['month'][0]:02d}-{dt['day'][0]:02d} {dt['hour'][0]:02d}:{dt['min'][0]:02d}:{dt['sec'][0]:02d}"
            for dt in date_time
        ]

    # Parse date_time
    o_date_time_parsed = parse_date_time(o_date_time)
    d_date_time_parsed = parse_date_time(d_date_time)

    # Create DataFrames
    o_df = pd.DataFrame({
        'date_time': o_date_time_parsed,
        'species': o_obs_species,
        'tilenum': o_obs_tilenum,
        'lon': o_obs_lon,
        'lat': o_obs_lat
    })

    d_df = pd.DataFrame({
        'date_time': d_date_time_parsed,
        'species': d_obs_species,
        'tilenum': d_obs_tilenum,
        'lon': d_obs_lon,
        'lat': d_obs_lat
    })

    # Merge DataFrames on all columns
    merged_df = pd.merge(o_df.reset_index(), d_df.reset_index(), on=['date_time', 'species', 'tilenum', 'lon', 'lat'], suffixes=('_o', '_d'))

    # Extract indices
    o_indices = merged_df['index_o'].values
    d_indices = merged_df['index_d'].values

    # Convert to numpy arrays
    o_indices = np.array(o_indices)
    d_indices = np.array(d_indices)

    # Apply the indices to the data arrays
    o_obs_obs = o_obs_obs[o_indices]
    o_obs_species = o_obs_species[o_indices]
    o_obs_fcst = o_obs_fcst[o_indices]
    o_obs_ana = o_obs_ana[o_indices]

    d_obs_obs = d_obs_obs[d_indices]
    d_obs_species = d_obs_species[d_indices]
    d_obs_fcst = d_obs_fcst[d_indices]
    d_obs_ana = d_obs_ana[d_indices]

    # Calculate the daily statistics in observation space
    for i in range(1, 14):
        o_omf = o_obs_fcst[o_obs_species == i] - o_obs_obs[o_obs_species == i]
        d_omf = d_obs_fcst[d_obs_species == i] - d_obs_obs[d_obs_species == i]
        b_omf = o_obs_fcst[o_obs_species == i] - d_obs_obs[d_obs_species == i]

        o_oma = o_obs_ana[o_obs_species == i] - o_obs_obs[o_obs_species == i]
        d_oma = d_obs_ana[d_obs_species == i] - d_obs_obs[d_obs_species == i]

        o_inc = o_obs_ana[o_obs_species == i] - o_obs_fcst[o_obs_species == i]
        d_inc = d_obs_ana[d_obs_species == i] - d_obs_fcst[d_obs_species == i]

        # Append the daily mean values to the daily mean lists
        daily_o_omf_mean.append(np.nanmean(o_omf))
        daily_d_omf_mean.append(np.nanmean(d_omf))
        daily_b_omf_mean.append(np.nanmean(b_omf))
        daily_o_oma_mean.append(np.nanmean(o_oma))
        daily_d_oma_mean.append(np.nanmean(d_oma))
        daily_o_inc_mean.append(np.nanmean(o_inc))
        daily_d_inc_mean.append(np.nanmean(d_inc))

    # Append the current date to the dates list
    dates_list.append(current_date.strftime('%Y%m%d'))

    # Increment the current date by one day
    current_date += timedelta(days=1)

In [ ]:
print('Length of daily_o_omf_mean: ', len(daily_o_omf_mean))

species_names = ['SMOS_fit_Tbh_A', 'SMOS_fit_Tbh_D', 'SMOS_fit_Tbv_A', 'SMOS_fit_Tbv_D', 'SMAP_L1C_Tbh_A', 
                 'SMAP_L1C_Tbh_D', 'SMAP_L1C_Tbv_A', 'SMAP_L1C_Tbv_D', 'ASCAT_META_SM', 'ASCAT_METB_SM', 
                 'ASCAT_METC_SM', 'MYD10C1', 'MOD10C1']

# Make daily time series plots of the mean OMF, OMA, and increments

# Convert the lists to numpy arrays
daily_o_omf_mean = np.array(daily_o_omf_mean)
daily_d_omf_mean = np.array(daily_d_omf_mean)
daily_b_omf_mean = np.array(daily_b_omf_mean)
daily_o_oma_mean = np.array(daily_o_oma_mean)
daily_d_oma_mean = np.array(daily_d_oma_mean)
daily_o_inc_mean = np.array(daily_o_inc_mean)
daily_d_inc_mean = np.array(daily_d_inc_mean)

# Convert the dates list to a numpy array
dates_list = np.array(dates_list)

# Plot the daily time series of the mean OMF, OMA, and increments for each species

for i in range(1, 14):

    plt.figure(figsize=(12,6))
    plt.plot(dates_list, daily_d_omf_mean[i-1::13])
    plt.plot(dates_list, daily_b_omf_mean[i-1::13])
    plt.plot(dates_list, daily_d_oma_mean[i-1::13])
    # Black dashed line at 0
    plt.axhline(y=0, color='k', linestyle='--')
    plt.legend(['DA OmF', 'bcOL OmF', 'DA OmA'])
    # xtick labels only on every 7th day
    plt.xticks(dates_list[::5], rotation=45)
    # Put species names in the title
    plt.title(f'{species_names[i-1]}')
    # Save the figure
    plt.savefig(f'/Users/amfox/Desktop/GEOSldas_diagnostics/figures/daily_time_series_{species_names[i-1]}.png')

for i in range(1, 14):

    plt.figure(figsize=(12,6))
    plt.plot(dates_list, daily_o_omf_mean[i-1::13])
    plt.plot(dates_list, daily_b_omf_mean[i-1::13])
    # Black dashed line at 0
    plt.axhline(y=0, color='k', linestyle='--')
    plt.legend(['OL OmF', 'bcOL OmF'])
    # xtick labels only on every 7th day
    plt.xticks(dates_list[::5], rotation=45)
    # Put species names in the title
    plt.title(f'{species_names[i-1]}')
    # Save the figure
    plt.savefig(f'/Users/amfox/Desktop/GEOSldas_diagnostics/figures/daily_time_series_OL_{species_names[i-1]}.png')



